In [8]:
import re

regex = (r"(?<=Answer:\s)(-?\d+(\.\d+)?)")
text = "Answer: 0b22fa63d0f54a529c525afbf2e8bb25"
action_match = re.search(regex, text)

tool_input = action_match.group().strip()


In [4]:
from huggingface_hub import hf_hub_download
import json
from tqdm import tqdm
import networkx as nx
from utils.data import load_hub_data, load_nav_graphs

    
# def load_graph(location_data, navigable_data, scans):
#     """
#     Load graph from scan,
#     Store the graph {scan_id: graph} in graphs
#     Store the shortest path {scan_id: {view_id_x: {view_id_y: [path]} } } in paths
#     Store the distances in distances. (Structure see above)
#     Load connectivity graph for each scan, useful for reasoning about shortest paths
#     """
    
#     graphs = load_nav_graphs(location_data, navigable_data, scans)

#     shortest_paths = {}
#     graphs_bar = tqdm(
#         graphs.items(),
#         desc='Computing shortest paths',
#     )
#     for scan, G in graphs_bar:  # compute all shortest paths
#         shortest_paths[scan] = dict(nx.all_pairs_dijkstra_path(G))

#     shortest_distances = {}
    
#     graphs_bar = tqdm(
#         graphs.items(),
#         desc='Computing shortest distances',
#     )
#     for scan, G in graphs_bar:  # compute all shortest paths
#         shortest_distances[scan] = dict(nx.all_pairs_dijkstra_path_length(G))
    
#     return graphs, shortest_paths, shortest_distances


In [10]:
import os
import pickle
from tqdm import tqdm
import networkx as nx

def load_graph(location_data, navigable_data, scans, cache_dir='../data/graph_cache'):
    os.makedirs(cache_dir, exist_ok=True)

    paths_path = os.path.join(cache_dir, 'shortest_paths.pkl')
    dists_path = os.path.join(cache_dir, 'shortest_distances.pkl')

    graphs = load_nav_graphs(location_data, navigable_data, scans)

    if all(os.path.exists(p) for p in [paths_path, dists_path]):
        print("Loading precomputed paths from cache...")
        with open(paths_path, 'rb') as f:
            shortest_paths = pickle.load(f)
        with open(dists_path, 'rb') as f:
            shortest_distances = pickle.load(f)
    else:
        print("Cache not found, computing from scratch...")

        shortest_paths = {}
        graphs_bar = tqdm(graphs.items(), desc='Computing shortest paths')
        for scan, G in graphs_bar:
            shortest_paths[scan] = dict(nx.all_pairs_dijkstra_path(G))

        shortest_distances = {}
        graphs_bar = tqdm(graphs.items(), desc='Computing shortest distances')
        for scan, G in graphs_bar:
            shortest_distances[scan] = dict(nx.all_pairs_dijkstra_path_length(G))

        print("Save shortest path")
        with open(paths_path, 'wb') as f:
            pickle.dump(shortest_paths, f)

        print("Save shortest distance")
        with open(dists_path, 'wb') as f:
            pickle.dump(shortest_distances, f)

    return graphs, shortest_paths, shortest_distances
